In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import seaborn as sns
import re
from IPython.core.display import HTML

In [2]:
palms = pd.read_csv('../data/palms_may6_vrbo.csv')

In [3]:
palms = palms.drop_duplicates('data.results.listings.listingId')

In [4]:
palms = palms.rename(columns={'data.results.listings.bedrooms':'bedrooms', 'data.results.listings.propertyType':'type',
       'data.results.listings.sleeps':'sleeps', 'data.results.listings.petsAllowed':'pets_allowed',
       'data.results.listings.averageRating':'rating',
       'data.results.listings.reviewCount':'review_count',
       'data.results.listings.bathrooms.full':'full_bathrooms',
       'data.results.listings.bathrooms.half':'half_bathrooms',
        'data.results.listings.prices.perNight.roundedFormattedAmount':'cost_night',
       'data.results.listings.geoCode.latitude':'latitude',
       'data.results.listings.geoCode.longitude':'longitude'})

In [5]:
palms = palms.drop(columns = {'data.results.listings.virtualTourBadge',
       'data.results.listings.listingId',
       'data.results.listings.detailPageUrl',
       'data.results.listings.instantBookable',
       'data.results.listings.listingNumber',
       'data.results.listings.propertyId',
        'data.results.listings.__typename',
        'data.results.listings.minStayRange.minStayHigh',
       'data.results.listings.minStayRange.minStayLow',
       'data.results.listings.minStayRange.__typename',
       'data.results.listings.propertyMetadata.headline',
       'data.results.listings.propertyMetadata.__typename',
       'data.results.listings.unitMetadata.unitName',
       'data.results.listings.unitMetadata.__typename',
        'data.results.listings.bathrooms.toiletOnly',
       'data.results.listings.bathrooms.__typename',
       'data.results.listings.spaces.__typename',
       'data.results.listings.spaces.spacesSummary.bedCountDisplay',
       'data.results.listings.spaces.spacesSummary.__typename',
       'data.results.listings.spaces.spacesSummary.area.areaValue',
       'data.results.listings.spaces.spacesSummary.area.__typename',
       'data.results.listings.geoDistance.text',
       'data.results.listings.geoDistance.relationType',
       'data.results.listings.geoDistance.__typename',
       'data.results.listings.priceSummary.priceAccurate',
       'data.results.listings.priceSummary.priceTypeId',
       'data.results.listings.priceSummary.edapEventJson',
       'data.results.listings.priceSummary.formattedAmount',
       'data.results.listings.priceSummary.roundedFormattedAmount',
       'data.results.listings.priceSummary.pricePeriodDescription',
       'data.results.listings.priceSummary.__typename',
       'data.results.listings.priceSummarySecondary.priceTypeId',
       'data.results.listings.priceSummarySecondary.edapEventJson',
       'data.results.listings.priceSummarySecondary.formattedAmount',
       'data.results.listings.priceSummarySecondary.roundedFormattedAmount',
       'data.results.listings.priceSummarySecondary.pricePeriodDescription',
       'data.results.listings.priceSummarySecondary.__typename',
       'data.results.listings.priceLabel.priceTypeId',
       'data.results.listings.priceLabel.pricePeriodDescription',
       'data.results.listings.priceLabel.__typename',
       'data.results.listings.prices.mainPrice',
       'data.results.listings.prices.__typename',
       'data.results.listings.prices.perNight.amount',
       'data.results.listings.prices.perNight.formattedAmount',
         'data.results.listings.prices.perNight.pricePeriodDescription',
       'data.results.listings.prices.perNight.__typename',
       'data.results.listings.prices.total.amount',
       'data.results.listings.prices.total.formattedAmount',
       'data.results.listings.prices.total.roundedFormattedAmount',
       'data.results.listings.prices.total.pricePeriodDescription',
       'data.results.listings.prices.total.__typename',
        'data.results.listings.geoCode.__typename',
       'data.results.listings.amenitiesBadges[0].id',
       'data.results.listings.amenitiesBadges[0].name',
       'data.results.listings.amenitiesBadges[0].helpText',
       'data.results.listings.amenitiesBadges[0].__typename',
       'data.results.listings.images[0].altText',
       'data.results.listings.images[0].c6_uri',
       'data.results.listings.images[0].c9_uri',
       'data.results.listings.images[0].mab',
       'data.results.listings.images[0].__typename',
       'data.results.listings.rankedBadges[0].id',
       'data.results.listings.rankedBadges[0].name',
       'data.results.listings.rankedBadges[0].helpText',
       'data.results.listings.rankedBadges[0].__typename',
       'data.results.listings.superlativesBadges[0].id',
       'data.results.listings.superlativesBadges[0].name',
       'data.results.listings.superlativesBadges[0].helpText',
       'data.results.listings.superlativesBadges[0].__typename',
       'data.results.listings.virtualTourBadge.id',
       'data.results.listings.virtualTourBadge.name',
       'data.results.listings.virtualTourBadge.helpText',
       'data.results.listings.virtualTourBadge.__typename',
       'data.results.listings.images[0].mab.banditId',
       'data.results.listings.images[0].mab.payloadId',
       'data.results.listings.images[0].mab.campaignId',
       'data.results.listings.images[0].mab.cached',
       'data.results.listings.images[0].mab.arm',
       'data.results.listings.images[0].mab.__typename',
       'data.results.listings.webRatingBadges[0].id',
       'data.results.listings.webRatingBadges[0].name',
       'data.results.listings.webRatingBadges[0].helpText',
       'data.results.listings.webRatingBadges[0].__typename', 'data.results.id',
       'data.results.propertyRedirectUrl', 'data.results.__typename',
       'data.results.resultCount', 'data.results.page',
       'data.results.pageSize', 'data.results.pageCount',
       'data.results.queryUUID', 'data.results.pinnedListing',
       'data.results.fromRecord', 'data.results.toRecord',
       'data.results.internalTools', 'data.results.typeaheadSuggestion.name',
       'data.results.typeaheadSuggestion.uuid',
       'data.results.typeaheadSuggestion.term',
       'data.results.typeaheadSuggestion.__typename',
       'data.results.geography.name', 'data.results.geography.lbsId',
       'data.results.geography.gaiaId', 'data.results.geography.isGeocoded',
       'data.results.geography.shouldShowMapCentralPin',
       'data.results.geography.__typename',
       'data.results.geography.description',
       'data.results.geography.primaryGeoType',
       'data.results.geography.location.latitude',
       'data.results.geography.location.longitude',
       'data.results.geography.location.__typename',
       'data.results.geography.breadcrumbs[0].name',
       'data.results.geography.breadcrumbs[0].countryCode',
       'data.results.geography.breadcrumbs[0].primaryGeoType',
       'data.results.geography.breadcrumbs[0].__typename',
       'data.results.geography.breadcrumbs[0].location.latitude',
       'data.results.geography.breadcrumbs[0].location.longitude',
       'data.results.geography.breadcrumbs[0].location.__typename',
       'data.results.destination.__typename',
       'data.results.destination.breadcrumbs[0].name',
       'data.results.destination.breadcrumbs[0].url',
       'data.results.destination.breadcrumbs[0].__typename',
       'data.results.destinationMessage.iconText',
       'data.results.destinationMessage.__typename',
       'data.results.destinationMessage.iconTitleText.title',
       'data.results.destinationMessage.iconTitleText.message',
       'data.results.destinationMessage.iconTitleText.icon',
       'data.results.destinationMessage.iconTitleText.messageValueType',
       'data.results.destinationMessage.iconTitleText.link',
       'data.results.destinationMessage.iconTitleText.__typename',
       'data.results.percentBooked.currentPercentBooked',
       'data.results.percentBooked.__typename', 'data.results.parsedParams.q',
       'data.results.parsedParams.sort',
       'data.results.parsedParams.__typename',
       'data.results.parsedParams.coreFilters.adults',
       'data.results.parsedParams.coreFilters.children',
       'data.results.parsedParams.coreFilters.pets',
       'data.results.parsedParams.coreFilters.minBedrooms',
       'data.results.parsedParams.coreFilters.maxBedrooms',
       'data.results.parsedParams.coreFilters.minBathrooms',
       'data.results.parsedParams.coreFilters.maxBathrooms',
       'data.results.parsedParams.coreFilters.minNightlyPrice',
       'data.results.parsedParams.coreFilters.maxNightlyPrice',
       'data.results.parsedParams.coreFilters.minSleeps',
       'data.results.parsedParams.coreFilters.__typename',
       'data.results.parsedParams.dates.arrivalDate',
       'data.results.parsedParams.dates.departureDate',
       'data.results.parsedParams.dates.__typename',
       'data.results.mapViewport.neLat', 'data.results.mapViewport.neLong',
       'data.results.mapViewport.swLat', 'data.results.mapViewport.swLong',
       'data.results.mapViewport.__typename',
       'data.results.globalMessages.alert',
       'data.results.globalMessages.__typename',
       'data.results.globalMessages.banner',
       'data.results.filterGroups[0].__typename',
       'data.results.filterGroups[0].groupInfo.id',
       'data.results.filterGroups[0].groupInfo.name',
       'data.results.filterGroups[0].groupInfo.__typename',
       'data.results.filterGroups[0].filters[0].count',
       'data.results.filterGroups[0].filters[0].checked',
       'data.results.filterGroups[0].filters[0].__typename',
       'data.results.filterGroups[0].filters[0].filter.id',
       'data.results.filterGroups[0].filters[0].filter.name',
       'data.results.filterGroups[0].filters[0].filter.refineByQueryArgument',
       'data.results.filterGroups[0].filters[0].filter.description',
       'data.results.filterGroups[0].filters[0].filter.__typename',
       'data.results.filterGroups[0].filters[0].filter.groupId',
        'data.results.listings.prices.label'})

In [6]:
palms['cost_night'] = palms['cost_night'].str.replace('$', '')

C:\Users\bswil\AppData\Local\Temp\ipykernel_4548\3685218847.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  palms['cost_night'] = palms['cost_night'].str.replace('$', '')


In [7]:
palms['cost_night'] = palms['cost_night'].str.replace(',', '')

In [8]:
palms['cost_night'] = palms['cost_night'].astype(int)

In [9]:
palms['half_bathrooms'] = palms['half_bathrooms']/2

In [10]:
palms['bathrooms'] = palms['full_bathrooms']+palms['half_bathrooms']

In [11]:
palms = palms.drop(columns={'full_bathrooms', 'half_bathrooms'})

In [12]:
palms['checkin_month'] = '5'
palms['checkin_day'] = '6'

In [13]:
palms['location'] = 'Isle of Palms'

In [14]:
palms

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,bathrooms,checkin_month,checkin_day,location
0,2,Villa,6,False,4.662162,74,256,32.806271,-79.736050,2.0,5,6,Isle of Palms
1,3,House,10,True,0.000000,0,331,32.795515,-79.773828,2.5,5,6,Isle of Palms
2,4,House,11,True,4.931035,87,520,32.779926,-79.802704,3.0,5,6,Isle of Palms
3,3,House,8,False,4.750000,20,319,32.804501,-79.744851,2.0,5,6,Isle of Palms
4,4,House,10,False,0.000000,0,544,32.802890,-79.737720,4.0,5,6,Isle of Palms
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,3,House,6,True,4.966667,60,353,32.825928,-79.815434,2.0,5,6,Isle of Palms
67,3,House,7,True,5.000000,12,304,32.825199,-79.817210,2.0,5,6,Isle of Palms
68,1,Studio,4,False,4.750000,8,306,32.760826,-79.840528,1.0,5,6,Isle of Palms
69,3,House,6,True,5.000000,1,350,32.803476,-79.771988,1.0,5,6,Isle of Palms
